# Compound Mirror

Author: A W Jones

This notebook checks the ABCD matrix propergation for a compound mirror (i.e an ITM) which is made up of two optical surfaces seperated by space with a high refractive index.

## Setup
The following equations are the analytic resposne and are copied from `pykat/optics/ABCD.py`. They match Chapter 9 in Living Reviews in Relativity.

In [1]:
import finesse
import finesse.gaussian as gaussian
from finesse import BeamParam
from finesse.utilities import refractive_index
import numpy as np

def apply(ABCD, q1, n1, n2):
    """
    Applied the matrix ABCD to a complex beam parameter
    
    ABCD: Component matrix
    q1: input beam parameter
    n1: input refractive index
    n2: output refractive index
    """
    # make sure we have a complex here. We could have a beamParam object or a an actual complex value supplied
    #_q1 = complex(q1)
        
    return BeamParam(nr=n2, q=n2 * (
        ABCD[0,0] * q1/float(n1) + ABCD[0,1]) / (ABCD[1,0] * q1/float(n1) + ABCD[1,1]),
        wavelength=q1.wavelength)

def mirror_trans(n1, n2, Rc):
    return np.matrix([[1.0,0.0],[(n2-n1)/float(Rc),1.0]])

def space(n,L):
    return np.matrix([[1.0, L/float(n)],[0.0,1.0]])

# Refractive indicies
nmedium = 3.451 # Silicon at 2u
nvac = 1 

print(f'Critical angle {180*np.arcsin(nvac/nmedium)/np.pi:.2f} deg')

Critical angle 16.84 deg


# Test 1 - Correct ABCD Matrix In/Out Matricies

In [2]:
Rc = 1

IFO = finesse.Model()
IFO.parse(f"""
mirror ITM_HR Rc={Rc}
space substrate portA=ITM_HR.p2 portB=ITM_AR.p1 nr={nmedium}
mirror ITM_AR
space lin portA=ITM_AR.p2 portB=PR3.p1 nr={nvac}
mirror PR3
""")

print(IFO.component_tree(IFO.ITM_HR))

# Check parsed ok
try:
    assert np.all(IFO.ITM_AR.Rc == [np.inf, np.inf])
except:
    print('AR surface not flat!')
    import sys
    sys.exit(10)
else:
    print('AR parsed ok')
    
# Check parsed ok
try:
    assert np.all(IFO.ITM_HR.Rc == [Rc, Rc])
except:
    print('HR surface not flat!')
    import sys
    sys.exit(10)
else:
    print('HR parsed ok')

print('\nDebug infomation:')
print(f'n outside (HR): {refractive_index(IFO.ITM_HR.p1)}')
print(f'n inside (HR): {refractive_index(IFO.ITM_HR.p2)}')
print('')

M_ana = mirror_trans(nmedium,nvac,np.inf)
M_fin = IFO.ABCD(IFO.ITM_AR.p1.i,IFO.ITM_AR.p2.o).M
try:
    assert np.all(
            np.isclose(M_ana,M_fin,atol=1e-15,rtol=1e-10))
except:
    print('Expected ABCD at AR surface: ')
    print(M_ana)
    print('Computed ABCD:')
    print(M_fin)
    import sys
    sys.exit(10)
else:
    print('AR matrix ok')
    
M_ana = mirror_trans(nvac,nmedium,Rc)
M_fin = IFO.ABCD(IFO.ITM_HR.p1.i,IFO.ITM_HR.p2.o).M
try:
    assert np.all(
            np.isclose(M_ana,M_fin,atol=1e-15,rtol=1e-10))
except:
    print('Expected ABCD at HR surface: ')
    print(M_ana)
    print('Computed ABCD:')
    print(M_fin)
    import sys
    sys.exit(10)
else:
    print('HR matrix ok')
    


○ ITM_HR
╰──○ ITM_AR
   ╰──○ PR3
AR parsed ok
HR parsed ok

Debug infomation:
n outside (HR): None
n inside (HR): 3.451

AR matrix ok
HR matrix ok


# Test 2 - Check Beam Trace

In [3]:
# Styled like the ligo file
IFO = finesse.Model()
IFO.parse_legacy(f"""

l IO_l 1 0.0 0.0 nLaser
s input_space 1 nLaser SC_ni

# 20 ppm loss as in LIGO
m2 SC_itmAR 0 20u 0 SC_ni SC_nSubAR
s SC_itmSub 30m $nmedium SC_nSubAR SC_nSubHR
m SC_itm 0.994 0.006 0.0 SC_nSubHR SC_n1
attr SC_itm Rcx {-Rc}
attr SC_itm Rcy {-Rc}

s Larm 1.5 SC_n1 SC_n2

m SC_etm 0.99999 1e-05 0.0 SC_n2 SC_no
attr SC_etm Rcx {Rc}
attr SC_etm Rcy {Rc}

cav SC SC_etm SC_n2 SC_itm SC_n1
const nmedium {nmedium}

yaxis abs
lambda 2e-06
""")

trace = IFO.beam_trace()

print(f'ITM P2 attached to {IFO.SC_itm.p2.attached_to}')
print(f'SC_itmAR P1 attached to {IFO.SC_itmAR.p1.attached_to}')

bpin = trace[IFO.SC_itm.p2.i].qx
print(f'bp itm in w: {1e3*bpin.w:.3f}mm Rc: {bpin.Rc}m')

            finesse.model [ WARNING]: Modal model enabled with only HG00. Call Model.select_modes to add modes.


ITM P2 attached to <'Larm' @ 0x7f311b50c580 (Space)>
SC_itmAR P1 attached to <'input_space' @ 0x7f311b549f70 (Space)>
bp itm in w: 1.050mm Rc: 1.0m


In [4]:
def check(expected,computed,name):
    """ Quick function to contain the common code """
    try:
        # check real physical numbers
        assert np.isclose(expected.w,computed.w,atol=1e-15,rtol=1e-10)
        assert np.isclose(expected.Rc,computed.Rc,atol=1e-15,rtol=1e-10)
    except:
        print(f'Expected beam radius after {name}: {1e3*expected.w} mm, computed: {1e3*computed.w}mm')
        print(f'Expected Rc after {name}: {expected.Rc}m, computed: {computed.Rc}m')
        print('Failed due to mismatch')
        import sys
        sys.exit(10)
    else:
        print(f'{name} ok')


_1 = apply(mirror_trans(1,nmedium ,Rc),bpin,1,nmedium)
check(_1,
    trace[IFO.SC_itm.p1.o].qx,
    'HR surface')

_2 = apply(space(nmedium ,30e-3),_1,nmedium,nmedium)
check(_2,
    trace[IFO.SC_itmAR.p2.i].qx,
    'subtrate space')

_3 = apply(mirror_trans(nmedium,1,np.inf),_2,nmedium ,1)
check(_3,
    trace[IFO.SC_itmAR.p1.o].qx,
    'AR surface')

HR surface ok
subtrate space ok
AR surface ok
